이 장에서는 FAC언어(\ref{chap:FunArithEval}장)에 단순타입을 추가한 FACT언어를 정의한다.
하지만, 단순타입 람다계산법(STLC)의 타입규칙을 따르면서 고정접 조합자를 작성할 수 없으므로
FACT언어에는 재귀함수를 표현하기 위한 별도의 문법 요소가 추가되어 있으며,
그에 따라 의미구조에도 재귀함수를 고려하는 규칙이 추가된다.
이렇게 타입이 추가된 언어의 문법규칙과 의미구조를 살펴본 다음,
논리학에서 모델이론과 증명이론의 개념이 프로그래밍어에서 의미구조와 타입규칙과
어떤 관련이 있는지 간단히 설명한다. 그리고는 FACT언어를 매개로 하여
타입검사와 타입유추의 개념을 알아본다.
\vspace*{5em}

In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

\begin{align*}
f,x &\;\in\; \texttt{Nm} \qquad\qquad n \;\in\; \mathbb{Z} \\
\uwave{\textcolor{blue}{\tau}\phantom{y}\!\!}\!\! &\,\uwave{~\textcolor{blue}{\in\, \texttt{Ty} ~::=~ \mathbb{Z} ~\mid~ \tau_1\!\to\tau_2}} \\
e &\;\in\, \texttt{Expr}
    ~::=~ x
    ~\mid~ \lambda x{:}\tau.e
    ~\mid~ \uwave{\textcolor{blue}{\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e)}}
    ~\mid~ e_1~e_2 \\
 & \qquad\qquad~\,
    ~\mid~ n ~\mid~ e_1 +\, e_2
    ~\mid~ \textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0
\\[-3em]
\end{align*}
\begin{align*}
\texttt{Env} &\;=\; \texttt{Nm} \xrightharpoonup{_\textrm{fin}} \texttt{Val} & ~~
\rho &\in \texttt{Env} \,::= \{x_1{\mapsto}v_1,\,\ldots,\,x_n{\mapsto}v_n\} \\
\texttt{Val} &\,\subset\, (\texttt{Expr}\times\texttt{Env}) \cup \mathbb{Z} & ~~
v &\in \texttt{Val} \,::= \langle \lambda x{:}\tau.e, \rho \rangle
                     \mid \uwave{\textcolor{blue}{\langle \textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e), \rho \rangle}}
                     \mid n \\
\cdot\,{\Longmapsto}\,\cdot &\,\subset\, (\texttt{Expr}\times\texttt{Env})\times\texttt{Val}
\\[-2em]
\end{align*}

\fbox{$\langle e,\rho\rangle \Longmapsto v$}$\qquad\qquad\langle x,\rho\rangle \Longmapsto \rho(x)$\vspace{-1ex}
$$
\langle\lambda x{:}\tau.e,\rho\rangle \Longmapsto \langle\lambda x{:}\tau.e,\rho\rangle
\qquad \quad
\uwave{\textcolor{blue}{\langle\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e),\rho\rangle \Longmapsto \langle\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e),\rho\rangle}}
$$
$$
\frac{\begin{array}{l}
      \langle e_1,\rho\rangle \Longmapsto \langle\lambda x{:}\tau.e,\rho_1\rangle \\
      \langle e_2,\rho\rangle \Longmapsto v_2 \\
      \langle e,\{x{\mapsto}v_2\}\rho_1\rangle \Longmapsto v
      \end{array}
    }{\langle e_1~e_2,\rho\rangle \Longmapsto v}
\qquad
\uwave{\textcolor{blue}{
\frac{\begin{array}{l}
      \langle e_1,\rho\rangle \Longmapsto \langle\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e),\rho_1\rangle \\
      \langle e_2,\rho\rangle \Longmapsto v_2 \\
      \langle e,\{x{\mapsto}v_2,\,f{\mapsto}\langle\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e),\rho_1\rangle\}\rho_1\rangle \Longmapsto v
      \end{array}
    }{\langle e_1~e_2,\rho\rangle \Longmapsto v}}}
$$
$$
\langle n,\rho\rangle \Longmapsto n
\qquad
\frac{\langle e_1,\rho\rangle \Longmapsto n_1 
      \quad
      \langle e_2,\rho\rangle \Longmapsto n_2}{
        \langle e_1 +\, e_2,\rho\rangle \Longmapsto n}(n\equiv n_1+n_2)
$$
$$
\frac{\langle e,\rho\rangle \Longmapsto 0
      \quad
      \langle e_0,\rho\rangle \Longmapsto n_0}{
        \langle\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0,\,\rho\rangle \Longmapsto n_0}
\quad        
\frac{\langle e,\rho\rangle \Longmapsto n
      \qquad
      \langle e_1,\rho\rangle \Longmapsto n_1}{
        \langle\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0,\,\rho\rangle \Longmapsto n_1}(n\neq 0)
$$

# 단순타입과 재귀함수요약식이 추가된 FACT언어

FACT언어의 문법구조와 의미구조(그림 \ref{fig:FACTlang})는 기본적으로
FAC언어의 문법구조와 의미구조(그림 \ref{fig:FAClang})를 거의 그대로 따르므로
차이점이 무엇인지에만 주목해 살펴보자. 참고로, FAC언어에는 없고 FACT에 추가된 내용을
그림 \ref{fig:FACTlang}에서 \uwave{\textcolor{blue}{파란색}}으로 표시하고 있다.

문법구조에서 드러나는 두 가지 차이점은 다음과 같다.
첫째, STLC의 람다요약식($\lambda x{:}\tau.e$)과 같이 매개변수($x$)의 타입($\tau$)을 명시한다.
둘째, 새로운 예약어 \textbf{rec}으로 시작하는 재귀함수요약식이 추가되었다.
FACT언어에서 재귀함수요약식 $\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e)$ 의미는
FAC언어에서 $Z$조합자(\ref{sec:Zcombinator}절)를 활용한 $Z\,(\lambda f.(\lambda x.e))$에 해당한다.
직관적으로는 $f$가 그 자신을 포함한 재귀함수요약식 $\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e)$을 대표한다고 이해할 수 있다.
즉, 함수 몸체($e$)에서 재귀함수를 호출하려는 곳에 $f$를 활용하면 된다.
예컨대, 0부터 자연수 $i$까지 합을 구하는 재귀함수를 
$\textbf{rec}\;f{:}\mathbb{Z}{\to}\mathbb{Z}\;(\lambda i{:}\mathbb{Z}.\;\textbf{if}\;i\;\textbf{then}\;i+f\,(i+(-1))\,\textbf{else}\;i)$로 작성한다.

의미구조에서는 람다요약식의 매개변수에 타입이 추가된 점을 제외하고는 사실상 기존의 의미규칙을 그대로 따르되,
재귀함수와 관련된 두 개의 의미규칙이 더 추가되었다. 첫째는 일반적인 함수요약식과 마찬가지로
재귀함수요약식도 더 이상 계산이 진행되지 않고 바로 클로저를 구성하여 값으로 취급하는 규칙이다.
둘째는 함수적용식($e_1\;e_2$)의 함수 부분($e_1$)를 계산한 결과값이 재귀함수요약식의 클로저인 경우를 처리하는 규칙이다.
재귀함수요약식의 클로저를 적용할 때는 보통 함수요약식의 클로저를 적용할 때처럼
매개변수($x$)를 인자 부분$(e_2$)의 결과값($v_2$)에 대응($x{\mapsto}v_2$)시킬 뿐만 아니라,
재귀함수를 대표하는 이름($f$) 또한 재귀함수요약식의 클로저에
대응($f{\mapsto}\langle\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e),\rho_1\rangle$)시켜
확장된 환경($\{x{\mapsto}v_2,f{\mapsto}\langle\ldots\rangle\}\rho_1$)에서 함수 몸체($e$)를 계산한다.

매번 재귀함수 호출이 일어날 때마다 환경에 재귀함수요약식의 클로저를 추가하는 동작방식이 잘못된 의미를 나타내는 것은 아니지만 이를 곧이곧대로 구현하는 것은 비효율적이다.
그럼에도 이렇게 비효율적인 방식으로 재귀함수에 대한 의미규칙을 작성하기도 하는 이유가 있기는 하다.
환경이나 클로저 등의 계산과정을 표현하는 매개체가 되는 구조에 대한 재귀적 표현에 의존하지 않고서도 재귀함수의 동작을 정의하는 의미규칙을 작성할 수 있음을 보여주기 때문이다.
하지만, 구현에 있어서는 재귀함수요약식 $\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e)$와 환경 $\rho$로 클로저를 만들 때부터
다음과 같이 자기 자신을 참조하는 바인딩을 환경에 포함하도록 클로저를 구성하는 것이 더 간편하다.

재귀함수요약식의 클로저를 이렇게 구성하면 그림 \ref{fig:FACTlang}에 정리된 의미구조에 비해 두 가지 측면에서 간소화된다.
값(`Val`)의 형태가 다시 두 종류로 단순해지며,
함수적용식($e_1~e_2$)에 대한 의미규칙도 하나만 있으면 된다.

정리하면, 환경에서 자기 자신을 참조하는 클로저를 만드는 방식의 재귀함수요약식에 대한 의미규칙을 아래와 같이 표현할 수 있다.
\vspace*{-1ex}
$$\textcolor{blue}{
\begin{array}{lcl}
\langle\textbf{rec}\,f{:}\tau\,(\lambda x{:}\tau_2.e),\rho\rangle \Longmapsto v
&~~\textrm{where}~~&
   v = \langle\lambda x{:}\tau_2.e,\rho'\rangle \\
&& \rho' = \{f{\mapsto}v\}\rho \\[-1ex]
\end{array} }$$
이 의미규칙 하나가 그림 \ref{fig:FACTlang}에 파란색으로 표시된 두 개의 재귀함수와 관련된 의미규칙을 대체한다.

# FACT언어의 인터프리터
환경에서 자신을 참조하는 재귀적 클로저를 활용하여 간소화된 의미규칙에 따라 FACT언어의 인터프리터를 하스켈로 작성해 보자.
우선, 문법구조를 나타내는 타입(`Ty`)과 식(`Expr`) 그리고 의미구조를 표현하기 위한 값(`Val`)과 환경(`Env`)을 아래와 같이 선언한다.

In [2]:
type Nm = String  -- 변수 이름은 문자열로
data Ty = Z | Arr Ty Ty  deriving (Eq,Ord)
data Expr = Var Nm             -- x
          | Lam Nm Ty Expr     -- (\x:t.e)
          | Rec Nm Ty Expr     -- rec f:t (\x:t2.e)
          | App Expr Expr      -- (e1 e2)
          | Lit Int            -- n
          | Add Expr Expr      -- e1 + e2
          | If Expr Expr Expr  -- if e then e1 else e0
          deriving (Eq, Ord)

data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]
type TEnv = [ (Nm, Ty) ] -- 타입환경

In [3]:
instance Show Ty where
  showsPrec _ Z = showString "\\mathbb{Z}"
  showsPrec p (Arr t1 t2) = showParen (p > 1) $
      showsPrec p t1 . showString "{\\to}" . showsPrec p t2
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x t e) = showParen (p > 1) $
      showString ("\\lambda "++x++"{:}"++show t++".") . showsPrec 1 e
  showsPrec p (Rec f t e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f++"{:}") . showsPrec 0 t .
      showString "\\, " . showsPrec 2 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0

import IHaskell.Display (latex)
dTeX e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Expr  where display = dTeX
instance IHaskellDisplay Ty    where display = dTeX

In [4]:
idExpr = Lam "x" Z (Var "x")
ttExpr = Lam "x" Z (Lam "y" Z (Var "x")) 
ffExpr = Lam "x" Z (Lam "y" Z (Var "y")) 
-- 람다식, 산술식과, 조건식이 모두 활용된 예시 프로그램
If (Add (Lit 3) (Var "y"))
   ffExpr
   (App idExpr (Rec "f" (Arr Z (Arr Z Z)) ffExpr))

In [5]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show TEnv where
    show tenv = "\\{"++ intercalate ",\\," (map show tenv) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,cl) = x++"{\\mapsto}"++show cl
instance {-# OVERLAPS #-} Show (Nm,Ty) where
    show (x,t) = x++"{\\mapsto}"++show t
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "
instance {-# OVERLAPS #-} IHaskellDisplay Env    where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay TEnv   where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val) where display=dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Ty)  where display=dTeX
instance IHaskellDisplay Val                     where display = dTeX

In [6]:
eval :: Env -> Expr -> Val  -- FACT언어의 인터프리터를 구현한 eval 함수
eval env (Var x) =
    case lookup x env of  Nothing -> error (x ++ " not defined")
                          Just v  -> v
eval env e@(Lam _ _ _) = Cl e env
eval env (Rec f t e@(Lam _ _ _)) = undefined -- 간소화된 의미규칙대로 작성
eval env (App e1 e2) =
    case v1 of  Cl (Lam x _ e) env1 -> eval ((x,v2):env1) e
                _    -> error (show v1++" not closure")
    where v1 = eval env e1 
          v2 = eval env e2 
eval _   (Lit n)     = Vi n
eval env (Add e1 e2) =
    case (v1, v2) of  (Vi n1, Vi n2) -> Vi (n1 + n2)
                      (Vi _ , _    ) -> error (show v2++" not int")
                      _              -> error (show v1++" not int")
    where v1 = eval env e1
          v2 = eval env e2
eval env (If e e1 e0) =
    case eval env e of  Vi 0 -> eval env e0
                        Vi _ -> eval env e1
                        _    -> error (show e++" not int")

FACT언어의 인터프리터 함수 `eval`을 위와 같이 같이 작성해 볼 수 있다.
재귀함수를 제외하면, 매개변수에 타입을 표기할 뿐이지 FAC언어와 실행 내용은 같다.
아래는 타입 표기만 추가된 \ref{chap:FunArithEval}장과 마찬가지 테스트의 실행 사례다.

In [7]:
e1 = Lam "x" Z . Lam "y" Z $ If (Var "x") (Var "x" `Add` Var "y")
                                          (Var "y" `Add` Var "y")
e2 = Lit 2
e3 = Lit 3

eval [] e1
eval [] e2

In [8]:
App e1 e2
eval [] (App e1 e2)

In [9]:
App (App e1 e2) e3
eval [] (App (App e1 e2) e3)

In [10]:
App (App e1 (Lit 0)) e3
eval [] (App (App e1 (Lit 0)) e3)

$~$\vspace*{-1em}\newline\indent
FACT언어에서는 0부터 주어진 자연수 $i$까지의 총합을 구하는 재귀함수를 아래와 같이 간단히 작성할 수 있다.

In [11]:
e4 = Rec "f" (Arr Z Z) .
         Lam "i" Z $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4 -- rec f (\i.if i then i + f(i + -1) else i)

\noindent
그런데, 아직 `eval`함수에는 재귀함수요약식에 대한 내용이 작성되어 있지 않고 `undefined`로 남겨져 있다.
그래서 다음을 실행하면 에러가 난다.

In [12]:
-- rec f (\i.if i then i + f(i + -1) else i) 100
eval [] (e4 `App` Lit 100) -- 100부터 0까지의 총합

: 

\noindent
연습문제로 재귀적인 구조의 클로저를 활용하는 간소화된 의미구조를 하스켈로 작성하여 인터프리터 함수 `eval`를 완성한 다음
아래와 같이 100까지의 합을 구하도록 위의 테스트를 다시 실행하여 예상대로 0부터 100까지의 합이 5050으로 정상적으로 계산되는지 확인해 보라.

# 타입규칙과 의미규칙의 관계
폴란드의 수학자/논리학자/철학자인 타르스키\cite{Tarski1956}가 자유변수가 포함된
형식언어의 의미를 어떻게 해석할 것인가에 대해 고민을 계기로 형식논리학에서
\index{모델이론|see{model theory}}\index{model theory|see{모델이론}}모델이론(model theory)이 발전하였다.
모델이론이 확립된 이후로는 명제(proposition) 혹은 논리 문장(logical sentence)의 의미(semantics)를 모델이론의 시각에서 설명하는 것이
보편화되었다. 그리고 기존의 공리와 추론규칙으로부터 정리를 이끌어내는
\index{증명이론|see{proof theory}}\index{proof theory|see{증명이론}}증명이론(proof theory)과
모델이론을 관련지어 하나의 형식언어에 관한 증명이론과 모델이론에서 판단들 사이의 관계에서
성립하는 것이 바람직하게 여겨지는 중요한 성질인 안전성(soundness)이나 완전성(completeness)과
같은 개념은 프로그래밍언어의 타입을 다루는 이론에서도 활용되고 있다.

모델이론은 각 단어의 의미에 따라 일상의 문장을 해석하는 것처럼 논리문장의
개별 구성요소 각각의 의미로부터 전체 문장의 의미를 찾으므로 논리체계가
제공하는 형식언어 본연의 의미를 해석한다고 볼 수 있다. 예를 들면 참/거짓의
진리값을 대표하는 대상들을 논리연결사 혹은 논리연산으로 연결하여 논리적 문장을
구성하는 명제논리의 경우를 생각해 보자. $n$개의 이름(혹은 논리변수)를 포함하는
논리적 문장의 의미는 $2^n$개의 경우의 수를 따지는
진리표로 알아볼 수 있다. 예를 들여, $x_1 \land \neg x_2 \lor x_3$에 대한 진리표는 다음과 같다. 

이런 진리표를 우리는 논리적 문장 혹은 명제의 자연스런 \`의미\'로 보통 받아들이고 있다.
한편, 논리적 문장의 모양새를 보고 진리표의 크기보다 더 적은 단계만에
논리적 문장을 판단할 수도 있는 \`증명\'이라는 방법도 우리는 알고 있다.
예컨대, $P \lor \neg P$의 형태이면 $P$가 아무리 복잡한 내용이라도 그 의미를 따로
분석할 필요 없이 (고전논리에서라면) 전체 문장을 참으로 판단해버릴 수 있다. 또한
$P \land \neg P$의 형태이면 전체 문장은 거짓이라고 판단해버릴 수 있다. 이렇듯,
모델이론에서 제시하는 원론적인 의미를 따라 문장을 이루는 모든 구성요소의 의미를
뜯어보지 않고도 문장의 모양새 혹은 구조적 특징에 따라 더 빠르게 판단할 수 있는
지름길을 제공하는 것이 증명의 역할이라 볼 수 있다. 이러한 증명을 위한 공리와
추론규칙 등을 정리한 것을 증명이론이라 부른다.

명제논리보다 복잡한 논리체계에서는 증명이론의 도움을 받지 않고서는 유한 시간 안에
판단을 하는 것조차 불가능한 경우도 많다. 자연수 및 그에 연산을 대상으로 하는
술어논리의 경우가 대표적이다. 에컨대, 짝수 자연수는 무한히 많다는 것을 모델이론에만
입각해 하나하나 따져보는 방식으로 계속해서 조금 더 큰 자연수를 아무리 많이 검사해 보더라도
무한히 많은 자연수를 다 확인하는 것은 애초에 불가능하다. 이런 논리체계를 위해 설계된
증명이론의 수학적귀납법이라는 추론규칙으로 무한히 많은 자연수에 대한 많은 성질을
유한한 단계만에 판단하는 증명이 가능하다. 프로그래밍언어의 의미구조와 타입규칙도
이와 비슷한 관계에 있다. 프로그래밍언어로 작성한 함수를 정수에 적용했을 때 항상 정수가
나온다는 성질을 의미구조에 따라 아무리 많은 정수로 테스트해 보더라도, 애초에 정수가 무한하기
때문에, 모든 경우를 다 확인해 볼 수는 없다. 하지만 타입규칙에 따라
$\mathbb{Z}\to\mathbb{Z}$라는 타입의 함수임이 판단된다면, 그 함수를 어떤 정수에 적용하든
계산이 무한히 진행되는 경우가 아닌 한 계산되는 결과값은 언제나 정수라는 증명이 되는 셈이다.

$$
\Gamma \vdash x : \Gamma(x)
\qquad
\frac{ x{:}\tau_2,\,\Gamma \vdash e : \tau 
    }{ \Gamma \vdash \lambda x{:}\tau_2.e : \tau_2\to\tau }
\qquad
\frac{ f{:}\tau_2{\to}\tau,\,\Gamma \vdash \lambda x{:}\tau_2.e : \tau_2\to\tau
    }{ \Gamma \vdash \textbf{rec}\,f{:}\tau_2{\to}\tau\,(\lambda x{:}\tau_2.e) : \tau_2\to\tau }
$$
$$
\Gamma \vdash n : \mathbb{Z}
\qquad
\frac{ \Gamma \vdash e_1 : \mathbb{Z} \quad~~
       \Gamma \vdash e_2 : \mathbb{Z}
    }{ \Gamma \vdash e_1 + e_2 : \mathbb{Z} }
\qquad\quad
\frac{ \Gamma \vdash e_1 : \tau_2\to\tau \quad~~
       \Gamma \vdash e_2 : \tau_2
    }{ \Gamma \vdash e_1\;e_2 : \tau }
$$

$$
\frac{ \Gamma \vdash e : \mathbb{Z} \quad~~
       \Gamma \vdash e_1 : \tau     \quad~~
       \Gamma \vdash e_0: \tau 
    }{ \Gamma \vdash \mathbf{if}\;e\;\mathbf{then}\;e_1\,\mathbf{else}\;e_0 : \tau }
$$

좀더 구체적으로 모델이론과 증명이론의 관점을 프로그래밍언어의 의미구조와 타입규칙에 적용해 보자.
$\mathcal{M};\rho \vdash P$는 의미구조에 해당하는 모델 $\mathcal{M}$에 따라 언어의
상수, 키워드, 연산자 등에 구조적으로 의미를 부여하고 환경 $\rho$에 따라 자유변수를
값으로 대응시켜 논리적 문장 $P$의 의미를 따져보면 참이라는 판단을 표현한 것이다.\footnote{캐나다
워털루 대학의 CS245 Logic and Computation 과목의 예전 강의자료에
정리된 모델이론의 표기 형식을 참고하였다. \url{https://cs.uwaterloo.ca/~plragde/cs245old}}
프로그래밍언어에 대한 논리적 문장 $P$는 $e \Longmapsto v$와 같은 형식으로 작성된다고 하자.
그렇다면 $\mathcal{M};\rho \models e \Longmapsto v$는 의미구조 $M$에 따라 환경 $\rho$를
참조하여 \`\`$e$를 계산한 결과값은 $v$이다\'\'라는 문장을 참으로 판단할 수
있다는 표현이다. 대략적으로, 큰걸음 의미구조 $\langle e,\sigma \rangle \Longmapsto v$를
모델이론의 관점으로는 $\mathcal{M};\rho \models e \Longmapsto v$로 표현한다.\footnote{이는
물론 그야말로 대략적인 설명이다. 모델이론은 동작과정 의미구조보다는 지시적 의미구조과 관련짓는
것이 더 적절하기 때문이다. 여기서는 결정적 동작과정 의미구조로 규정되는 비교적 간단한 언어만을
고려한다고 가정하면 \ref{sec:bigStepArith}절에서 산술식의 사례에서처럼 큰걸음 의미구조의 정의와
거의 비슷한 구조를 따르는 지시적 의미구조가 존재한다고 가정하고 정상종료하지 않는 계산의 경우는
일단 고려하지 않는 상황에서의 대략적인 비유에 가까운 설명이다.
참고로, 모델이론을 실용적으로 응용하는 모델검증과 같은 분야에서는 프로그램의 실행 과정을
프로세스로 보고 프로세스의 관찰가능한 행동 혹은 이벤트를 모델로 삼는 동적논리(dynamic logic)나
시제논리(temporal logic)라는 논리체계를 활용하기도 하는데, 더 자세한 내용은
이 책의 범위를 벗어나는 전문적인 주제이다.}

한편, 모델이론의 관점에서 표현한 의미구조 $\mathcal{M};\rho \models e \Longmapsto v$에
대응되는 타입을 판단하는 증명이론 관점의 표현을 $\mathcal{M}';\Gamma \vdash e : \tau$의
형태로 작성할 수 있다. 이 때, $\mathcal{M}'$는 의미구조를 반영하는 $\mathcal{M}$를 요약한 모델,
즉 요약된 의미구조(\ref{sec:TypeAsAbsSem}절)라고도 볼 수 있는
타입규칙(그림 \ref{fig:FACTtyrules})을 반영하며, $\rho$와 $\Gamma$
그리고 $v$와 $\tau$는 다음을 조건을 만족한다.\vspace*{-1.5ex}

* $\rho(x')=v'$이면 $\Gamma(x')=\tau'$이고 $v' : \tau'$이다. (이를 간단히 $\rho:\Gamma$라고 쓰자.)
* $v : \tau$이다.\vspace*{-1.5ex}

\noindent
이 때, $v:\tau$는 $v$가 정수 $n$의 형태일 때는 $\vdash n:\tau$를,
$v$가 클로저 $\langle e',\rho'\rangle$형태일 때는 $\rho':\Gamma'$인
$\Gamma'$를 활용한 타입 판단 $\Gamma\vdash e':\tau$를 나타낸다.
즉, 환경 $\rho$에서 이름에 값을 대응시키는 대신 그 값을 요약한 타입에 대응시켜 요약한 타입환경이 $\Gamma$이며,
$e$를 계산한 결과값 $v$를 요약한 타입이 $\tau$라는 말이다. 의미구조와 타입규칙이 명확히 규정되어 있어 대응되는
모델 $\mathcal{M}$이나 $\mathcal{M}'$를 혼동할 여지가 없으면 생략하여
$\rho \models e \Longmapsto v$나 $\Gamma \vdash e : \tau$라고 표기한다.
모델이론과 증명이론에서 유래한 $\models$ 및 $\vdash$ 표기를 바탕으로
타입 시스템의 안전성(soundness)과 완전성(completeness)이라는 개념(\ref{ssec:StaticDynamicTy}절)을
간략하게 정리해 볼 수 있다.\vspace{-2ex}

\paragraph{타입 안전성}
  $\Gamma \vdash e : \tau$이면,
  $\rho:\Gamma$인 환경 $\rho$에서 $\rho \models e \Longmapsto v$이고 $v:\tau$이거나
  또는 환경 $\rho$에서 진행하는 $e$의 계산이 아예 종료되지 않는다.\vspace*{-2ex}

\paragraph{타입 완전성}
  $\rho \models e \Longmapsto v$이면,
 $\rho:\Gamma$인 타입환경 $\Gamma$에서
$\Gamma \vdash e : \tau$이고 $v:\tau$이다.\vspace*{1.5ex}

타입 안정성은 이상적인 정적 타입 언어가 추구하는 성질이다. 실제로 FACT언어는 타입 안전하다.
하지만 더 복잡한 언어는 타입 안정성을 추구하되 예외적인 상황을 고려해야 하는 경우가 발생한다.
FACT언어에 나눗셈 연산을 추가하면 어떻게 될지 생각해 보라. 나누는 수가 0인 오류를 타입 규칙으로
실행 전에 미리 모두 걸러내기는 쉽지 않기 때문에, 나누는 수가 0인 오류가 나는 경우를 제외하고
타입 안전하다는 식의 예외 조항을 달아 타입 안전성을 언급하곤 한다.
반면, 튜링완전한 정적 타입 범용언어는 애초부터 완전성을 만족하기는 일반적으로 불가능하다.
프로그래밍언어의 타입 시스템만이 아니라 원래 이런 개념이 나온 형식논리학에서도
일정 수준 이상의 기능을 포함한 무모순적 논리체계는 완전할 수 없음이
쿠르트 괴델의 불완전성 정리로 밝혀져 있다.

여기서는 타입 안전성의 증명을 논하기보다는 비정상 종료 사례를 살펴보자.
에러가 나던 \ref{chap:FunArithEval}장의 예시에 상응하는 FACT 식도 마찬가지 에러가 난다.

In [13]:
Var "x"  -- 자유변수 x
eval [] (Var "x")

: 

In [14]:
Lit 3 `App` Lit 1  -- 3을 1에 함수처럼 적용
eval [] (Lit 3 `App` Lit 1)

: 

In [15]:
Lit 3 `Add` (Lam "x" Z $ Var "x")  -- 3과 함수요약식의 덧셈
eval [] (Lit 3 `Add` (Lam "x" Z $ Var "x"))

: 

\noindent
위의 세 식에 대해서는 타입규칙을 만족하는 적절한 타입을 찾을 수 없다
(즉, 어떤 타입 $\tau$에 대해서도 $\nvdash e : \tau$).
이러한 식을 실행해 보기 전에 타입규칙에 맞는 적절한 타입이 존재하는지
여부를 확인할 수만 있다면, 위와 같이 비정상 종료되는 모든 FACT언어의 식을
실행 전에 타입규칙에 의거해 걸러낼 수 있음을 타입 안전성이 보장한다.

FACT언어에서 타입 완전성의 반례로는 아래와 같은 조건식을 들 수 있다.

In [16]:
If (Lit 0) (Lam "x" Z $ Var "x") (Lit 3)
eval [] (If (Lit 0) (Lam "x" Z $ Var "x") (Lit 3))

\noindent
분명히 3으로 정상 종료되지만 타입규칙에 맞는 적절한 타입을 찾을 수 없다.
왜냐하면 $\vdash (\lambda x{:}\mathbb{Z}.x):\mathbb{Z}\to\mathbb{Z}$인 반면
$\vdash 3:\mathbb{Z}$으로 참인 경우와 거짓인 경우에 해당하는 식의 타입이 
$\mathbb{Z}\to\mathbb{Z}$와 $\mathbb{Z}$로 서로 다르기 때문이다.
그림 \ref{fig:FACTtyrules}의 조건식에 대한 타입규칙은 조건이 참과 거짓일 때의
두 식 $e_1$과 $e_0$의 타입이 $\tau$로 같아야 한다는 전제조건 있으며,
이를 만족할 경우에만 전체 조건식에 알맞은 타입이 $\tau$라고 판단한다.
이렇듯 정상 종료되는 모든 식을 수용하지 못한다는 점에서 타입규칙이 불완전하다고 말한다.
이론적 한계로 인해 정적 타입 언어가 타입 완전성을 추구할 수 없다고 해서
잘 돌아가는 프로그램을 무턱대고 거부하도록 설계하지는 않는다.
비록 완전하지는 못할지언정 프로그래머의 편의를 위해 안전성을 되도록 해지치 않으면서
더 많은 문제없이 잘 실행되는 프로그램을 허용하려 노력하며 설계한다.
앞서 \ref{chap:ParaPoly}장에서 살펴본 다형성(polymorphsm)도
더 유연한 언어 설계를 위해 정적 타입 시스템에서 지원하는 대표적인 기능이다.

# 타입검사 함수 작성 시도
\label{sec:tryFACTcheck}
타입환경($\Gamma$)에서 FACT언어의 식($e$)이 특정 타입($\tau$)인지, 즉 $\Gamma \vdash e : \tau$인지
검사하는 함수 `check`을 타입규칙(그림 \ref{fig:FACTtyrules})에 따라 하스켈로 작성하려 한다면
아래와 같이 시도해 볼 수 있을 것이다.

In [17]:
check :: TEnv -> Expr -> Ty -> Bool
check tenv (Var x)      t  = lookup x tenv == Just t
check tenv (Lam x t2 e) (Arr t2' t)
                           = t2==t2' && check ((x,t2):tenv) e t
check tenv (Rec f t' e) t  = t == t' && check ((f,t'):tenv) e t
check tenv (App e1 e2)  t  = undefined -- 이 부분을 어떻게 작성할까?
check tevv (Lit _)      Z  = True
check tenv (Add e1 e2)  Z  = check tenv e1 Z && check tenv e2 Z
check tenv (If e e1 e0) t  = check tenv e Z && check tenv e1 t
                                            && check tenv e0 t
check _   _             _  = False

다른 부분을 작성하는 데는 크게 막힘이 없지만 `undefined`로 남겨놓은
함수적용식(`App e1 e2`)을 어떻게 검사해야할지가 문제이다. 타입규칙(그림 \ref{fig:FACTtyrules})에 따르면
`e1`의 타입은 함수 타입인 `Arr t2 t`이고
`e2`의 타입이 `t2`인지 검사하면 된다.
그러니까 `undefined`의 자리에 대략 다음과 같은 코드가 와야 할 것이다.\vspace*{1.5ex}

```haskell
  check tenv (App e1 e2)  t  = check tenv e1 (Arr t2 t) && check e2 t2
```

$~$\vspace*{-2ex}\newline
문제는 위 코드에서 `e2`의 타입인 `t2`가
정의되지 않은 하스켈 변수이므로 하스켈 컴파일러가 에러를 낸다는 점이다.
`t`는 함수 파라메터로 주어져 있지만 `t2`는 하스켈 프로그램에서 정의되지 않은 자유변수이기 때문이다.
그러니까, `e2`의 타입에 해당하는 `t2`가 무엇인지 추측해서, 즉 타입유추로
구체적으로 어떤 타입인지 확정해야 타입검사를 제대로 진행할 수 있을 것이다.
이렇듯, 타입유추보다 더 간단한 듯 보였던 타입검사 함수를 작성하려고 시도해 보니
사실은 타입유추가 필요하다는 것을 알게 되었다.

가장 단순무식한 타입유추로는 가능한 모든 타입
$\mathbb{Z}$, $\mathbb{Z}\to\mathbb{Z}$, $\mathbb{Z}\to\mathbb{Z}\to\mathbb{Z}$, $(\mathbb{Z}\to\mathbb{Z})\to\mathbb{Z}$, $\ldots$를 타입규칙에 맞는 타입이 나올 때까지
시도해 보는 방법을 생각해 볼 수 있다.
연습문제로 가능한 모든 타입(`Ty`)을 빠짐도 없고 중복도 없이 나열(enumerate)하는 무한한 하스켈 리스트를 정의해 보라.
이 단순무식하게 나열하는 타입유추 방식의 문제점은, 단지 알맞은 타입을 찾는 데 많은 시간이 걸릴 수 있어 비효율적일 뿐 아니라,
애초에 알맞은 타입이 없는 경우에는 (어떤 다른 기준으로 더 이상 시도하지 말라는 한계를 정해주지 않는다면)
무한히 많은 시도를 하느라 타입유추 프로그램이 종료되지 않는다는 점이다.

# 타입유추
타입환경 $\Gamma$와 FACT언어의 식 $e$가 주어졌을 때
$\Gamma \vdash e : \tau$를 만족하는 타입($\tau$)이 존재한다면
구체적으로 $\tau$가 무엇인지 계산하여 이를 포함하는 리스트(`[`$\tau$`]`)를 결과값으로,
존재하지 않는다면 빈 리스트(`[]`)를 결과값으로
항상 유한한 시간 안에 계산하는 타입유추 함수 `infer`를 작성하는 것이 가능하다.
연습문제로 FACT언어의 타입규칙(그림 \ref{fig:FACTtyrules})에 충실하게 아래 `infer` 함수의 정의을 완성해 보라.
즉, 주어진 $\Gamma$와 $e$에 대해 다음 조건을 만족하도록 작성하라는 말이다.\vspace*{-1.5ex}

 * `infer` $\Gamma\phantom{e}e$ `==` `[`$\tau\!\!$ `]`이면 $\Gamma\vdash e:\tau$이며, 그 역도 성립한다.
 * `infer` $\Gamma\phantom{e}e$ `==` `[]`이면 $\Gamma\vdash e:\tau$인 $\tau$는 존재하지 않으며, 그 역도 성립한다.\vspace*{-1ex}

In [18]:
check tenv e t = infer tenv e == [t]  -- check는 infer로 간단히 정의 가능

infer :: TEnv -> Expr -> [Ty]  -- 적절한 타입이 존재하지 않으면 빈 리스트
infer tenv (Var x)      = case lookup x tenv of Just t  -> [t]
                                                Nothing -> []
infer tenv (Lam x t2 e) = undefined
infer tenv (Rec f t e)  = undefined
infer tenv (App e1 e2)  = undefined
infer tevv (Lit _)      = [Z]
infer tenv (Add e1 e2)  = [Z | check tenv e1 Z, check tenv e2 Z]
infer tenv (If e e1 e0) = undefined

\noindent
이런 조건에 맞게 작성된 `infer` 함수로 타입규칙에 맞는 타입이 존재하는지 확인된 경우를
즉 `infer` $\Gamma\phantom{e}e$ `==` `[`$\tau\!\!$ `]`인 경우,
FACT언어의 타입 안전성에 따라 $\rho:\Gamma$를 만족하는 $\rho$로
`eval` $\rho$ $e$를 계산하면 $v:\tau$인 결과값 $v$가 계산되던가
끝나지 않는 계산이 진행된다. 참고로, 이렇게 타입규칙에 맞는 타입이
존재하는 식을 \`\`타입이 잘 맞는다\'\'(well-typed)고 한다.

타입 안전성이 있으면 인터프리터를 더 효율적으로 작성할 수 있다.
타입이 잘 맞지 않는 식은 실행에 앞서 걸러내고 타입이 잘 맞는 식만
실행하는 FACT 언어의 인터프리터 `ev`는 아래와 같이 `eval`에
비해 간결하게 작성할 수 있으며 따져야 하는 경우의 수가 적으므로
그만큼 더 효율적으로 실행된다. `[6]`번 셀의 `eval`처럼 실행 오류가
날만한 상황인지 검사할 필요가 없다. 왜냐하면, 계산을 정상적으로
진행하기 위해 필요한 형태의 값이 부분식으로부터 (끝나지 않는 계산만 아니라면)
항상 제대로 계산됨을 타입 안전성이 보장하기 때문이다.

In [19]:
ev :: Env -> Expr -> Val  -- 타입이 잘 맞는 식만 실행한다고 가정
ev env (Var x) = v
    where Just v = lookup x env
ev env e@(Lam _ _ _) = Cl e env
ev env (Rec f t e@(Lam _ _ _)) = undefined -- eval의 경우와 같이
ev env (App e1 e2) = ev ((x,v2):env1) e
    where Cl (Lam x _ e) env1 = ev env e1 
          v2                  = ev env e2 
ev _   (Lit n)     = Vi n
ev env (Add e1 e2) = Vi (n1 + n2)
    where Vi n1 = ev env e1
          Vi n2 = ev env e2
ev env (If e e1 e0) = if n/=0 then ev env e1
                              else ev env e0
    where Vi n = ev env e

타입이 잘 맞는 식인지 실행 전에 확인하는 기능을 포함하는,
닫힌식에 대한 인터프리터 `eval'`는 다음과 같이 `infer`과 `ev`를 활용해 정의할 수 있다.

In [20]:
eval' :: Expr -> [Val]  -- 타입 검사가 실패하면 빈 리스트
eval' e = [ev [] e | t <- infer [] e]

\section*{요점정리}
* 정적 타입을 지원하는 튜링완전한 범용 언어를 설계하기 위해서는
  반복이나 재귀를 표현하는 언어 요소를 별도로 제공해야 한다.
  왜냐하면 타입없는 람다계산법(UTLC)에서는 표현 가능하던 고정점 조합자(fixpoint combinator)의
  핵심적인 요소인 자기 자신에 대한 적용(self-application)에 알맞은 타입을 부여하기 어렵기 때문이다.
* FAC언어는 단순타입 람다계산법(STLC)을 기반으로 재귀함수요약식을 추가한 FACT언어의
  의미구조를 정의의할 때, 함수요약식에 대한 클로저와 별도로 재귀함수요약식에 대한 클로저를
  또 하나의  새로운 값(value)의 형태로 포함하도록 정의할 수 있다. 이렇게 정의하면
  추론규칙은 재귀적일지언정 추론규칙이 다루는 대상은 순환적이지 않으므로 의미구조에
  대한 이론적 분석에 유리하다.
* 구현의 효율성과 편의를 고려한 간소화된 FACT언어의 의미구조에서는
  재귀함수요약식의 값을 일반적인 함수요약식에 대한 값인 클로저와 기본적으로 같은
  구조이지만 환경에서 자기 자신의 클로저를 참조하는 순환적 구조의 클로저를
  허용하는 방식으로 정의할 수도 있다. 간소화된 의미구조에서는 새로운 종류의 값의 형태를
  별도로 고려할 필요도 없으며 재귀함수요약식만을 위한 별도의 추론규칙의 개수도 줄어든다.
* 타입규칙과 의미규칙의 관계는 형식논리학에서 형식논리의 증명이론과 모델이론의 관계에 해당한다고 볼 수 있다.
  형식논리 문장(프로그램)의 성질을 판단하는 데 있어,
  모델 이론(의미규칙)에 따라 일일이 실행해 보아서는 무한히 많은 경우를 고려해야 하므로 판단 불가능하거나
  유한한 경우만 고려하여 판단 가능하더라도 이를 판단하는 데 너무나 많은 계산 비용이 들어 비효율적인 경우가 많으므로,
  증명 이론(혹은 타입 시스템)에서는 문장(프로그램)의 모든 구성요소를 따져보지 않더라도 전체적인 모양새(혹은 구조적인 특징)를
  살펴보고 유한한 단계만에 더 효율적으로 그런 성질의 판단이 가능한 지름길을 제공하려는 한다.
* 안전성(soundness)과 완전성(completeness)는 같은 형식논리에 대한 모델 이론과 증명 이론의 관계에 대한 성질이다.
  안전성이란 증명이론의 지름길로 참이라고 판한한 내용이 결코 모델이론의 판단에서 벗어나지 않는 성질이다.
  반면, 완전성이란 모델이론에서 참이라고 판단하는 바를 모두 증명이론에서도 참이라고 판단 가능함을 뜻한다.
* 이상적인 정적 타입 언어는 타입 안전성를 추구하지만,
  튜링완전한 정적 타입 언어는 타입 완전성(completeness)을 만족시키는 것이 불가능한 이론적 한계가 있다.
  (이 이론적 한계는 그 유명한 괴델의 불완전성 정리, 그리고 같은 문제를 튜링머신을 통해 기술한
  정지 문제(halting problem)의 판단 불가능함과 맞닿아 있다.)
  그렇다고 해서 잘 돌아가는 프로그램을 무턱대고 거부하게 프로그래밍언어를 설계하지는 않으며,
  안전성을 해치치 않으면서 되도록 많은 정상적으로 실행되는 프로그램을 받아들이도록 유연한 타입 시스템을 설계하려고 노력한다.
* 단순타입 람다계산법(STLC)를 포함하는 FACT언어의 타입규칙에 따라 타입 검사 함수를 작성하려 시도
  타입 적용 부분에서 막힌다. 왜냐하면 더 어려운 문제로 보이는 타입유추 기능이 필요하기 때문이다.
* 처치 방식(Curch-style) 단순타입 람다계산법을 기반으로 하는 FACT언어에서는 함수요약식의 파라메터마다 빠짐없이
  그 타입을 표기하기 때문에, 값이 확정되지 않은 변수 이름의 타입이 무엇인지 모두 파악된다.
  따라서 FACT언어에 대한 타입유추를 함수를 작성하는 것은 그리 어렵지 않다.
* 타입 안전성을 만족하면, 타입검사 혹은 타입유추가 성공한 알맞은 타입을 찾을 수 있는 프로그램이 비정상 종료되지 않음이 보장된다.
  즉, $e:\tau$로 판단된 프로그램 $e$를 실제로 실행하면 정상적으로 종료되며 $v:\tau$인 값을 계산하거나
  아니면 아예 끝나지 않는 계산이 진행된다.
* 타입 안전성을 만족하면 언어 구현(인터프리터 혹은 컴파일러)에서 프로그램의 부분식이 계산 결과가 기대하는 형태가
  아니라서 (즉, 기대하는 타입의 값이 아니라서) 비정상 종료될지 걱정하며 검사할 필요가 없어지므로
  구현이 간소회되며, 기대하는 형태가 아닌지 맞는지 따져야 할 경우의 수가 적어지므로 더 효율적으로
  프로그램을 실행할 수 있다.

$~$
\section*{연습문제}
1. `[5]`번 셀의 `eval` 함수에서 `undefined`로 남겨놓은 부분을 완성한 다음
   (이후에 `[19]`번의 `ev` 함수에도 같은 내용 작성),
   `[12]`번 셀의 테스트를 다시 실행해 정상적인 결과가 나오는지 확인해 보라. 
1. `[13]`-`[15]`번 셀의 오류가 나는 FACT언어의 식들에 대해 타입규칙에 따라 적절한 타입을 찾을 수 없는 이유를 타입규칙에 의거해 각각 설명해 보라.
1. 모든 타입(`Ty`)을 빠짐 없고 중복 없이 나열(enummerate)하는 하스켈 리스트를 작성하라.
   (힌트: 1개의 $\mathbb{Z}$로 만들어지는 타입, 2개의 $\mathbb{Z}$로 만들어지는 타입, 3개의 $\mathbb{Z}$로 만들어지는 타입, $\ldots$
   과 같은 순서로 리스트를 계속해서 만들어 나가는 방법으로 접근해 보라.)
1. `[18]`번 셀의 `infer` 함수를 FACT언어의 타입규칙에 따라 작성하라.
1. 작성된 함수 `infer`를 `[13]`-`[16]`번 셀의 식에 적용하면 어떤 결과가 나오는지 확인해 보라.
1. `infer`로 0부터 주어진 자연수까지 합을 구하는 함수인 `[11]`번 셀의 `e4`의 타입이 무엇인지 알아보라.
1. `infer`로 타입을 찾을 수는 있지만 `eval`로 실행해 보면 계산이 끝나지 않는 사례를 찾아보라.
1. 주어진 환경 $\rho$에 대해 $\rho:\Gamma$를 만족하는 타입환경 $\Gamma$를 계산하는 하스켈
   함수 `env2tenv :: Env -> TEnv`를 작성하라. (힌트: 필요하다면 `infer` 활용) 
1. `[20]`번 셀의 `eval'`로 이번 장에서 다룬 타입이 잘 맞지 않는 식과 맞는 식을 포함한 여러 가지 예시를 실행해 보라.

\section*{탐구과제}
1. C와 같은 언어들에서는 \ref{sec:tryFACTcheck}절에서 `check`에 해당하는 내용의 프로그램을
   작성하더라도 함수 적용(혹은 함수 호출)의 타입을 검사하는 부분에서 FACT언어의 경우와 같은
   문제가 발생하지 않는다. 어째서 그런지 알아보고 FACT언어와 어떤 차이 때문에 그렇게 되는지
   설명해 보라.
1. `infer`과 `eval'`이 빈 리스트가 아니라 `eval`처럼 에러 메시지를 내기 위해서는 어떻게
   하스켈 코드를 수정해야 할지 생각해 보라. 또한 `eval`처럼 비정상 종료 `error` 함수가 아닌
   정상 종료되지만 타입유추가 성공한 경우와 구분되는 에러 메시지를 결과값으로 내려면 어떻게
   해야 할지도 알아보라.